# Byte Pair Encoding, BPE
---

기계에게 아무리 많은 단어를 학습시켜도 세상의 모든 단어를 알려줄 수는 없습니다. 만약 기계가 모르는 단어가 등장하면 그 단어를 단어 집합에 없는 단어란 의미에서 해당 ```토큰을 UNK(Unknown Token)```라고 표현합니다.  

이와 같이 모르는 단어로 인해 발생하는 문제를 ```OOV(Out-Of-Vocabulary) 문제```라고 합니다.

Subword Segmentation 작업은 하나의 단어는 더 작은 단위의 의미있는 여러 서브워드들의 조합으로 구성된 경우가 많기 때문에, 하나의 단어를 여러 Subword로 분리해서 단어를 인코딩 및 임베딩 하겠다는 의도를 가진 전처리 작업입니다.  이를 통해 OOV나 희귀 단어, 신조어와 같은 문제를 완화시킬 수 있습니다.

실제로 언어의 특성으로인해 영어나 한국어는 Subword Segmentation을 했을 때 어느정도 의미있는 단위로 나누는 것이 가능합니다. 이러한 작업을 하는 토크나이저를 Subword Tokenizer라고 합니다.

## 1. BPE(Byte Pair Encoding)
---
아래와 같은 문자열이 주어졌을 때 BPE을 수행한다고 해봅시다.  
```python
aaabdaaabac
```
BPE은 기본적으로 연속적으로 가장 많이 등장한 byte pair를 찾아서 하나의 글자로 병합하는 방식을 수행합니다.  
위 문장에서 가장 자주 등장하는 byte pair는 'aa'입니다. 이 'aa'라는 byte pair를 'Z'로 치환해보겠습니다.

```python
ZabdZabac
Z = aa
```

한 번 치환한 문자열에서 가장 많이 등장하는 byte pair는 'ab'입니다. 이 'ab'를 'Y'로 치환해봅시다.

```python
ZYdZYac
Y = ab
Z = aa
```

이번에 가장 많이 등장하는 byte pair는 'ZY'입니다. 이를 'X'로 치환해봅시다.
```python
XdXac
X = ZY
Y = ab
Z = aa
```

더 이상 병합할 byte pair가 없으므로 BPE는 위의 결과를 최종 결과로 하여 종료됩니다.


## 2. 자연어 처리에서의 BPE

논문 : https://arxiv.org/pdf/1508.07909.pdf

자연어 처리에서 BPE는 subword segmentation 알고리즘입니다. 요약하자면, character 단위에서 점차적으로 단어 집합(vocabulary)를 만들어 내는 bottom up 방식의 접근을 사용합니다.

먼저 훈련 데이터에 있는 단어들을 모든 character또는 unicode 단위로 단어 집합(vocabulary)를 만들고, 가장 많이 등장하는 유니그램을 하나의 유니그램으로 통합합니다.

1. BPE 알고리즘 사용

기존 딕셔너리에 아래와 같이 저장되어 있다고 하겠습니다.

```python
# dictionary
dict = {'low' : 5, 'lower' : 2, 'newest' : 6, 'widest' : 3}
```

우선 딕셔너리의 모든 단어들을 글자(character)단위로 분리합니다.

```python
l o w : 5, l o w e r : 2, n e w e s t : 6, w i d e s t : 3
```
딕셔너리를 참고로 한 초기 단어 집합(vocabulary)은 아래와 같습니다. 간단히 말해 초기 구성은 글자 단위로 분리된 상태입니다.

```python
l, o, w, e, r, n, s, t, i, d
```
BPE의 특징은 알고리즘의 동작을 몇 회 반복(iteration)할 것인지를 사용자가 정한다는 것입니다.

여기서는 총 10회를 수행한다고 가정하겠습니다. 이는 가장 빈도수가 높은 유니그램의 쌍을 하나의 유니그램으로 통합하는 과정을 10회 반복한다는 말입니다.

- 1회 : 딕셔너리를 참고 하였을 때 빈도수가 9로 가장 높은 (e, s)의 쌍을 es로 통합합니다.
```python
# dictionary update!
l o w : 5,
l o w e r : 2,
n e w es t : 6,
w i d es t : 3
```

```python
# vocabulary update!
l, o, w, e, r, n, s, t, i, d, es
```

- 2회 : 빈도수가 9로 가장 높은 (es, t)의 쌍을 est로 통합합니다.
```python
# dictionary update!
l o w : 5,
l o w e r : 2,
n e w est : 6,
w i d est : 3
```

```python
# vocabulary update!
l, o, w, e, r, n, s, t, i, d, es, est
```

- 3회 : 빈도수가 7로 가장 높은 (l, o)의 쌍을 lo로 통합합니다.
```python
# dictionary update!
lo w : 5,
lo w e r : 2,
n e w est : 6,
w i d est : 3
```

```python
# vocabulary update!
l, o, w, e, r, n, s, t, i, d, es, est, lo
```

지금까지와 같은 방식으로 총 10회 반복하였을 때의 딕셔너리와 단어 집합은 아래와 같습니다.

```python
# dictionary
low : 5,
low, e, r : 2,
newest : 6,
widest : 3
```

```python
# vocabulary
l, o, w, e, r, n, s, t, i, d, es, est, lo, low, ne, new, newest, wi, wid, widest
```

이 경우 테스트 과정에서 'lowest'라는 단어가 등장한다면, 기존에는 OOV에 해당하는 단어가 되었겠지만 BPE 알고리즘을 사용한 위의 단어 집합에서는 더 이상 'lowest'는 OOV가 아닙니다. 기계는 우선 'lowest'를 전부 글자 단위로 분할하고 'low'와 'est'를 찾아냅니다. 기계는 'lowest'를 'low'와 'est'두 단어로 인코딩 합니다.

## 2. 코드 실습

In [3]:
import re, collections
from IPython.display import display, Markdown, Latex

# BPE 수행 횟수
num_merges = 10

# dictionary
# </w>는 단어 맨 끝에 붙이는 특수 문자
dictionary = {'l o w </w>' : 5,
              'l o w e r </w>' : 2,
              'n e w e s t </w>' : 6,
              'w i d e s t </w>' : 3}

# BPE algorithm
def get_stats(dictionary):
    # 유니그램 pair들의 빈도수 카운트
    pairs = collections.defaultdict(int)
    for word, freq in dictionary.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i], symbols[i+1]] += freq
    print('현재 pair들의 빈도수 : ', dict(pairs))
    return pairs

def merge_dictionary(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

bpe_codes = {}
bpe_codes_reverse = {}

for i in range(num_merges):
    display(Markdown("### Iteration  {}".format(i+1)))
    pairs = get_stats(dictionary)
    best = max(pairs, key=pairs.get)
    dictionary = merge_dictionary(best, dictionary)

    bpe_codes[best] = i
    bpe_codes_reverse[best[0] + best[1]] = best

    print("new merge : {}".format(best))
    print("dictionary : {}".format(dictionary))


### Iteration  1

현재 pair들의 빈도수 :  {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 8, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('e', 's'): 9, ('s', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3}
new merge : ('e', 's')
dictionary : {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 6, 'w i d es t </w>': 3}


### Iteration  2

현재 pair들의 빈도수 :  {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'es'): 6, ('es', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'es'): 3}
new merge : ('es', 't')
dictionary : {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est </w>': 6, 'w i d est </w>': 3}


### Iteration  3

...

현재 pair들의 빈도수 :  {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('new', 'est</w>'): 6, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge : ('new', 'est</w>')
dictionary : {'low </w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration  9

현재 pair들의 빈도수 :  {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge : ('low', '</w>')
dictionary : {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration  10

현재 pair들의 빈도수 :  {('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge : ('w', 'i')
dictionary : {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}


In [4]:
# merge 기록 확인하기
print(bpe_codes)

{('e', 's'): 0, ('es', 't'): 1, ('est', '</w>'): 2, ('l', 'o'): 3, ('lo', 'w'): 4, ('n', 'e'): 5, ('ne', 'w'): 6, ('new', 'est</w>'): 7, ('low', '</w>'): 8, ('w', 'i'): 9}


## OOV 대처하기

In [5]:
def get_pairs(word):
    """
    Return set of symbol pairs in a word
    Word is represented as a tuple of symbols
    (symbols being variable-length strings)
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs

def encode(orig):
    """Encode word based on list of BPE merge operations, which are applied consecutively"""

    word = tuple(orig) + ('</w>',)
    display(Markdown("__word split into characters:__ <tt>{}</tt>".format(word)))

    pairs = get_pairs(word)

    if not pairs:
        return orig
    
    iteration = 0
    while True:
        iteration += 1
        display(Markdown("__Iteration {}:__".format(iteration)))

        print("bigrams in the word: {}".format(pairs))
        bigram = min(pairs, key = lambda pair : bpe_codes.get(pair, float('inf')))
        print("condidate for merging : {}".format(bigram))

        if bigram not in bpe_codes:
            display(Markdown("__Candidate not in BPE merges, algorithm stops.__"))
            break

        first, second = bigram
        new_word = []
        i = 0
        while i < len(word):
            try:
                j = word.index(first, i)
                new_word.extend(word[i:j])
                i = j
            except:
                new_word.extend(word[i:])
                break

            if word[i] == first and i < len(word) -1 and word[i+1] == second:
                new_word.append(first + second)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        
        new_word = tuple(new_word)
        word = new_word
        print("word after merging: {}".format(word))
        if len(word) == 1:
            break
        else:
            pairs = get_pairs(word)
    
    # </w> 는 출력 안함
    if word[-1] == '</w>':
        word = word[:-1]
    elif word[-1].endswith('</w>'):
        word = word[:-1] + (word[-1].replace('</w>', ''),)
    
    return word

In [6]:
encode('loki')

__word split into characters:__ <tt>('l', 'o', 'k', 'i', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('k', 'i'), ('o', 'k'), ('l', 'o'), ('i', '</w>')}
condidate for merging : ('l', 'o')
word after merging: ('lo', 'k', 'i', '</w>')


__Iteration 2:__

bigrams in the word: {('k', 'i'), ('lo', 'k'), ('i', '</w>')}
condidate for merging : ('k', 'i')


__Candidate not in BPE merges, algorithm stops.__

('lo', 'k', 'i')

In [7]:
encode('lowest')

__word split into characters:__ <tt>('l', 'o', 'w', 'e', 's', 't', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('l', 'o'), ('t', '</w>'), ('o', 'w'), ('e', 's'), ('s', 't'), ('w', 'e')}
condidate for merging : ('e', 's')
word after merging: ('l', 'o', 'w', 'es', 't', '</w>')


__Iteration 2:__

bigrams in the word: {('l', 'o'), ('t', '</w>'), ('o', 'w'), ('es', 't'), ('w', 'es')}
condidate for merging : ('es', 't')
word after merging: ('l', 'o', 'w', 'est', '</w>')


...

bigrams in the word: {('lo', 'w'), ('w', 'est</w>')}
condidate for merging : ('lo', 'w')
word after merging: ('low', 'est</w>')


__Iteration 6:__

bigrams in the word: {('low', 'est</w>')}
condidate for merging : ('low', 'est</w>')


__Candidate not in BPE merges, algorithm stops.__

('low', 'est')

In [8]:
encode('lowing')

__word split into characters:__ <tt>('l', 'o', 'w', 'i', 'n', 'g', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('g', '</w>'), ('l', 'o'), ('i', 'n'), ('n', 'g'), ('o', 'w'), ('w', 'i')}
condidate for merging : ('l', 'o')
word after merging: ('lo', 'w', 'i', 'n', 'g', '</w>')


__Iteration 2:__

bigrams in the word: {('g', '</w>'), ('i', 'n'), ('n', 'g'), ('lo', 'w'), ('w', 'i')}
condidate for merging : ('lo', 'w')
word after merging: ('low', 'i', 'n', 'g', '</w>')


__Iteration 3:__

bigrams in the word: {('g', '</w>'), ('n', 'g'), ('i', 'n'), ('low', 'i')}
condidate for merging : ('g', '</w>')


__Candidate not in BPE merges, algorithm stops.__

('low', 'i', 'n', 'g')

# WordPiece Tokenizer

- 논문 : https://static.googleusercontent.com/media/research.google.com/ko//pubs/archive/37842.pdf

- 구글이 위 WordPiece Tokenizer를 변형하여 번역기에 사용했다는 논문 : https://arxiv.org/pdf/1609.08144.pdf

WordPiece Tokenizer는 BPE의 변형 알고리즘입니다. 해당 알고리즘은 병합되었을 때 코퍼스의 우도(Likelihood)를 가장 높이는 쌍을 반환합니다.



## 학습 알고리즘

접두사를 추가하여 subwords를 식별하기 때문에 각 단어는 처음에 해당 접두사를 단어 내부의 모든 문자에 추가하여 분할됩니다. 예를 들어 'word'는 다음과 같이 분할됩니다.
```python
'w' '##o' '##r' '##d'
```

따라서 초기 알파벳에는 단어의 시작 부분에 있는 모든 문자들과 WordPiece 접두사가 선행하는 단어 내부에 있는 문자가 포함됩니다.

그런 다음 BPE와 마찬가지로 WordPiece도 병합 규칙을 학습합니다. 주요 차이점은 병합할 쌍이 선택되는 방식입니다. 가장 빈번하게 출현하는 쌍을 선택하는 대신 WordPiece는 다음 공식을 사용하여 각 쌍에 대한 점수를 계산합니다.

$$\text{score} = \frac{\text{freq of pair}}{(\text{freq of first element}) \times (\text{freq of second element})}$$

쌍의 빈도는 각 부분의 빈도의 곱으로 나눔으로써, 알고리즘은 각 개별 부분들의 빈도가 낮은 쌍의 병합에 높은 우선순위를 부여합니다.

예를 들어 vocabulary 내에서의 출현 빈도가 높은 ('un', '##able')쌍을 굳이 병합할 필요는 없는데, 그 이유는 'un'과 '##able'각각이 다른 단어 내에서 매무 빈번하게 출현하여 높은 빈도를 나타내기 때문입니다.

반면에 'hu'와 '##ging'은 각각이 자주 사용되지 않기 때문에 ('hu', '##gging')과 같은 쌍은 아마도 더 빨리 병합될 것입니다.

단어와 빈도수의 예시를 가지고 설명 하겠습니다.
```python
('hug', 10), ('pug', 5), ('pun', 12), ('bun', 4), ('hugs', 5)
```

분할 결과는 아래와 같습니다.
```python
('h', '##u', '##g', 10), ('p', '##u', '##g', 5), ('p', '##u', '##n', 12), ('b', '##u', '##n', 4), ('h', '##u', '##g', '##s', 5)
```

따라서 초기 vocabulary는 ```['b', 'h', 'p', '##g', '##n', '##s', '##u']```가 됩니다.

가장 빈번한 쌍은 ('##u', '##g')(현재 20회)이지만 '##u'의 개별 빈도가 매우 높아 점수가 가장 높지는 않습니다.(1/36)

'##u'가 포함된 모든 쌍은 실제로 동일한 점수를 가지므로 가장 좋은 점수는 ('##g', '##s')이 가지고 있습니다. 이는 '##u'가 없는 유일한 쌍입니다. 그리고 학습된 첫 번째 병합은 ('##g', '##s') -> ('##gs')입니다.

```python
Vocabulary : ['b', 'h', 'p', '##g', '##n', '##s', '##u', '##gs']
Corpus : ('h', '##u', '##g', 10), ('p', '##u', '##g', 5), ('p', '##u', '##n', 12), ('b', '##u', '##n', 4), ('h', '##u', '##gs', 5)
```

이 시점에서 '##u'는 가능한 모든 쌍에 있으므로 모두 동일한 점수를 가집니다. 이 경우 첫 번째 쌍이 병합되므로 ('h', '##u') -> 'hu'가 학습됩니다.

```python
Vocabulary : ['b', 'h', 'p', '##g', '##n', '##s', '##u', '##gs', 'hu']
Corpus : ('hu', '##g', 10), ('p', '##u', '##g', 5), ('p', '##u', '##n', 12), ('b', '##u', '##n', 4), ('hu', '##gs', 5)
```
이제 최고 점수는 ('hu', '##g') 및 ('hu', '##gs')가 동일하게 계산되므로 가장 큰 점수를 가진 첫 번째 쌍이 병합됩니다.

```python
Vocabulary : ['b', 'h', 'p', '##g', '##n', '##s', '##u', '##gs', 'hu', 'hug']
Corpus : ('hug', 10), ('p', '##u', '##g', 5), ('p', '##u', '##n', 12), ('b', '##u', '##n', 4), ('hu', '##gs', 5)
```

## 토큰화 알고리즘

토큰화는 WordPiece가 학습된 병합 규칙은 제외하고 최종 Vocabulary만 저장한다는 점에서 BPE와 다릅니다. 토큰화할 단어에서 시작하여 WordPiece는 Vocabulary에 있는 가장 긴 하위 단어를 찾은 다음 분할합니다.

예를 들어, 위의 예에서 학습한 vocabulary를 사용하는 경우, 단어 'hugs'의 경우는 처음부터 시작하는 가장 긴 하위 단어는 vocabulary 내부에 있는 'hug'이므로 ['hug', '##s']로 분할됩니다. 그런 다음 '##s'가 vocabulary에 존재하고 이를 계속 사용할 수 있으므로 'hugs'의 토큰 결과는 ['hug', '##s']입니다.

## WordPice 구현

In [9]:
corpus = [
    "This is the Hugging Face course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

c:\Users\Myeong\anaconda3\envs\jin\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Myeong\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

AttributeError: 'BertTokenizerFast' object has no attribute 'save'

In [10]:
# 말뭉치에 단어 빈도 계산
from collections import defaultdict

word_freqs = defaultdict(int)
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

word_freqs

defaultdict(int,
            {'This': 3,
             'is': 2,
             'the': 1,
             'Hugging': 1,
             'Face': 1,
             'course': 1,
             '.': 4,
             'chapter': 1,
             'about': 1,
             'tokenization': 1,
             'section': 1,
             'shows': 1,
             'several': 1,
             'tokenizer': 1,
             'algorithms': 1,
             'Hopefully': 1,
             ',': 1,
             'you': 1,
             'will': 1,
             'be': 1,
             'able': 1,
             'to': 1,
             'understand': 1,
             'how': 1,
             'they': 1,
             'are': 1,
             'trained': 1,
             'and': 1,
             'generate': 1,
             'tokens': 1})

In [11]:
alphabet = []
for word in word_freqs.keys():
    if word[0] not in alphabet:
        alphabet.append(word[0])
    for letter in word[1:]:
        if f"##{letter}" not in alphabet:
            alphabet.append(f"##{letter}")

alphabet.sort()
alphabet

['##a',
 '##b',
 '##c',
 '##d',
 '##e',
 '##f',
 '##g',
 '##h',
 '##i',
 '##k',
 '##l',
 '##m',
 '##n',
 '##o',
 '##p',
 '##r',
 '##s',
 '##t',
 '##u',
 '##v',
 '##w',
 '##y',
 '##z',
 ',',
 '.',
 'F',
 'H',
 'T',
 'a',
 'b',
 'c',
 'g',
 'h',
 'i',
 's',
 't',
 'u',
 'w',
 'y']

In [13]:
vocab = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"] + alphabet.copy()

splits = {
    word : [c if i == 0 else f"##{c}" for i,c in enumerate(word)] for word in word_freqs.keys()
}

In [14]:
def compute_pair_scores(splits):
    letter_freqs = defaultdict(int)
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            letter_freqs[split[0]] += freq
            continue
        for i in range(len(split)-1):
            pair = (split[i], split[i+1])
            letter_freqs[split[i]] += freq
            pair_freqs[pair] += freq
        letter_freqs[split[-1]] += freq
    
    scores = {
        pair : freq / (letter_freqs[pair[0]] * letter_freqs[pair[1]]) for pair, freq in pair_freqs.items()
    }
    return scores

pair_scores = compute_pair_scores(splits)
for i, key in enumerate(pair_scores.keys()):
    print(f"{key} : {pair_scores[key]}")
    if i >= 5:
        break

('T', '##h') : 0.125
('##h', '##i') : 0.03409090909090909
('##i', '##s') : 0.02727272727272727
('i', '##s') : 0.1
('t', '##h') : 0.03571428571428571
('##h', '##e') : 0.011904761904761904


In [17]:
best_pair = ''
max_score = None
for pair, score in pair_scores.items():
    if max_score is None or max_score < score:
        best_pair = pair
        max_score = score

print(best_pair, max_score)

('a', '##b') 0.2


따라서 학습할 첫 번째 병합은 ('a', '##b') -> 'ab'이고 vocabulary에 'ab'를 추가합니다.

In [18]:
vocab.append('ab')

def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue
        i = 0
        while i < len(split) -1:
            if split[i] == a and split[i+1] == b:
                merge = a + b[2:] if b.startswith("##") else a + b
                split = split[:i] + [merge] + split[i+2 :]
            else:
                i += 1
        splits[word] = split
    return splits

splits = merge_pair('a', '##b', splits)
splits['about']

['ab', '##o', '##u', '##t']

In [19]:
vocab_size = 70
while len(vocab) < vocab_size:
    scores = compute_pair_scores(splits)
    best_pair, max_score = '', None
    for pair, score in scores.items():
        if max_score is None or max_score < score:
            best_pair = pair
            max_score = score
    splits = merge_pair(*best_pair, splits)
    new_token = (
        best_pair[0] + best_pair[1][2:] if best_pair[1].startswith("##") else best_pair[0] + best_pair[1]
    )
    vocab.append(new_token)

In [20]:
print(vocab)

['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', '##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##k', '##l', '##m', '##n', '##o', '##p', '##r', '##s', '##t', '##u', '##v', '##w', '##y', '##z', ',', '.', 'F', 'H', 'T', 'a', 'b', 'c', 'g', 'h', 'i', 's', 't', 'u', 'w', 'y', 'ab', '##fu', 'Fa', 'Fac', '##ct', '##ful', '##full', '##fully', 'Th', '##hm', '##thm', 'Hu', 'Hug', 'Hugg', 'ch', 'cha', 'chap', 'chapt', 'sh', 'th', 'is', '##thms', '##za', '##zat', '##ut', '##ta']


In [21]:
def encode_word(word):
    tokens = []
    while len(word) > 0:
        i = len(word)
        while i > 0 and word[:i] not in vocab:
            i -= 1
        if i == 0:
            return ["[UNK]"]
        tokens.append(word[:i])
        word = word[i:]
        if len(word) > 0:
            word = f"##{word}"
    return tokens

print(encode_word("Hugging"))
print(encode_word("HOgging"))

['Hugg', '##i', '##n', '##g']
['[UNK]']


In [22]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    encoded_words = [encode_word(word) for word in pre_tokenized_text]
    return sum(encoded_words, [])

tokenize('This is the Hugging Face course!')

['Th',
 '##i',
 '##s',
 'is',
 'th',
 '##e',
 'Hugg',
 '##i',
 '##n',
 '##g',
 'Fac',
 '##e',
 'c',
 '##o',
 '##u',
 '##r',
 '##s',
 '##e',
 '[UNK]']

# Unigram Language Model Tokenizer

- 논문 : https://arxiv.org/pdf/1804.10959.pdf

유니그램 언어 모델 토크나이저는 각각의 subword들에 대해서 loss를 계산합니다. 측정된 subword들의 loss를 정렬하여, 최악의 영향을 주는 10~20%의 토큰을 제거합니다. 이를 원하는 단어 집합의 크기에 도달할 때까지 반복합니다.

## 학습 알고리즘

크기가 큰 vocabulary에서 시작하여 원하는 vocabulary 크기에 도달할 때까지 토큰을 제거합니다. 

학습의 각 단계에서 Unigram알고리즘은 현재 vocabulary가 주어졌을 때의 말뭉치에 대한 loss를 계산합니다. 그런 다음, vocabulary의 각 symbol에 대해, 해당 기호가 제거되면 전체 손실이 얼마나 증가할지 계산하고 가장 적게 증가하는 symbol을 찾습니다. 이렇게 찾은 symbols는 말뭉치에 대한 전체 손실에 더 적은 영향을 미치므로 어떤 의미에서는 '덜 필요(less needed)'하고 제거 대상으로 가장 적합한 후보입니다.

이것은 비용이 많이 드는 작업이므로 가장 낮은 손실을 초래하는 기호 하나만 제거하지 않고, 이러한 기호들의 $p\%$를 제거합니다.

설명에는 위 예제의 말뭉치를 재사용합니다.
```python
('hug', 10), ('pug', 5), ('pun', 12), ('bun', 4), ('hugs', 5)
```

초기 vocabulary는 위 말뭉치에 존재하는 모든 단어들의 모든 하위 문자열(substrings)로 구성됩니다.
```python
['h', 'u', 'g', 'hu', 'ug', 'p', 'pu', 'n', 'un', 'b', 'bu', 's', 'hug', 'gs', 'ugs']
```

## 토큰화 알고리즘

Unigram 모델은 개별 토큰들의 출현 분포가 서로 독립적(i.i.d)이라는 가정을 하는 언어 모델 유형입니다. 토큰 X의 확률이 문맥에 상관없이 동일하다는 점에서 가장 단순한 언어 모델입니다. 따라서 Unigram 언어 모델을 사용하여 텍스트를 생성하는 경우 항상 가장 일반적이고 흔한(common)토큰을 도출합니다.

특정 토큰의 확률은 말뭉치 내에서의 해당 토큰 출현 빈도를 vocabulary에 존재하는 모든 토큰들의 출현 빈도의 합으로 나눈 것입니다.

다음은 vocabulary에서의 모든 하위 단어(subwords)의 빈도입니다.
```python
("h", 15) ("u", 36) ("g", 20) ("hu", 15) ("ug", 20) ("p", 17) ("pu", 17) ("n", 16)
("un", 16) ("b", 4) ("bu", 4) ("s", 5) ("hug", 15) ("gs", 5) ("ugs", 5)
```

따라서 모든 빈도의 합은 210이고 subword 'ug'의 확률은 20/210입니다.

이제 주어진 단어를 토큰화하기 위해, 가능한 모든 토큰 분할을 살펴보고 Unigram 모델에 따라 각각의 확률을 계산합니다. 모든 토큰의 출현 빈도가 독립적인 것으로 간주되기 때문에 이 확률은 각 토큰의 확률의 곱일 뿐입니다. 예를 들어, 'pug'의 토큰화 결과인 ['p', 'u', 'g']의 확률은 다음과 같이 계산됩니다.

$$P([\text{"p"},\; \text{"u"},\; \text{"g"}]) = P(\text{"p"}) \times P(\text{"u"}) \times P(\text{"g"}) = \frac{5}{210} \times \frac{36}{210} \times \frac{2}{210} = 0.000389$$

이에 비해 토큰화 결과인 ["pu", "g"]의 확률은 다음과 같습니다.

$$P([\text{"pu"},\; \text{"g"}]) = P(\text{"pu"}) \times P(\text{"g"}) = \frac{5}{210} \times \frac{20}{210} = 0.0022676$$

따라서, ["pu", "g"]이 훨씬 더 자주 출현한다고 볼 수 있습니다. 일반적으로 가장 적은 수의 하위 토큰들로 구성된 토큰화 결과는 비교적 높은 확률을 가지며, 이는 우리가 직관적으로 원하는 결과입니다.

Unigram 모델을 사용한 단어의 토큰화는 가장 높은 확률을 나타내는 분할 형태로 토큰화됩니다. 'pug'의 예에서 가능한 각 분할에 대해 얻을 수 있는 확률은 다음과 같습니다.
```python
['p', 'u', 'g'] : 0.000389
['p', 'ug'] : 0.0022676
['pu', 'g'] : 0.0022676
```

위의 경우에는 가능한 모든 분할을 찾고 확률을 계산하는 것이 쉬웠지만, 일반적으로는 더 어려울 수 있습니다. 이를 위해 사용되는 알고리즘인 **Viterbi 알고리즘**이 있습니다.

기본적으로 주어진 단어에 대한 가능한 모든 분할들을 나타낼 수 있는 그래프를 구성할 수 있습니다. 그리고 만일 주어진 단어 내의 문자 a에서 b까지의 하위 단어(subword)가 vocabulary에 존재한다면, 우리는 이 그래프 내에서 a에서 출발하여 b까지 가는 그래프 내의 가지(branch)가 있다고 말할 수 있습니다. 그리고 이 하위 단어의 확률을 해당 가지(branch)에 지정할 수 있습니다.

그래프에서 최고 점수를 얻을 경로를 찾기 위해 Viterbi 알고리즘은 단어 내의 각 위치에 대해 해당 위치에서 끝나는 경로의 최고 점수를 나타내는 분할을 결정합니다. 단어의 처음 위치부터 끝까지 이동하면서, 현재 위치에서 끝나는 모든 하위 단어를 검사한 다음 이 하위 단어가 시작하는 위치에서 최고의 토큰화 점수를 사용하여 최상의 점수를 찾을 수 있습니다. 그런 다음 끝에 도달하기 위해 선택한 경로를 펼치기만 하면 됩니다.

```python
Character 0 (u): "u" (score 0.171429)
Character 1 (n): "un" (score 0.076191)
Character 2 (h): "un" "h" (score 0.005442)
Character 3 (u): "un" "hu" (score 0.005442)
Character 4 (g): "un" "hug" (score 0.005442)
```

## 학습 알고리즘 2

이제 토큰화가 어떻게 작동하는지 보았으므로 학습 과정에서 사용된 loss에 대해 조금 더 자세히 알아보겠습니다. 각각의 주어진 단계에서 이 loss는 말뭉치 내의 모든 단어를 토큰화하여 계산됩니다. 계산 과정에서 앞에서 설명한 것처럼 현재 vocabulary와 말뭉치에 있는 각 토큰의 빈도에 의해 결정된 유니그램 모델을 사용합니다.

말뭉치의 각 단어별로 점수를 계산하며 손실은 해당 점수의 negative log likelihood입니다. 즉, 말뭉치에 있는 모든 단어의 $-\log (P(\text{word}))$합입니다.

위에서 설명한 말뭉치를 가지고 설명하겠습니다.
```python
("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)
```

각 단어의 토큰화 결과 및 점수는 다음과 같습니다.
```python
"hug": ["hug"] (score 0.071428)
"pug": ["pu", "g"] (score 0.007710)
"pun": ["pu", "n"] (score 0.006168)
"bun": ["bu", "n"] (score 0.001451)
"hugs": ["hug", "s"] (score 0.001701)
```

이때의 loss는 다음과 같습니다.
```python
10 * (-log(0.071428)) + 5 * (-log(0.007710)) + 12 * (-log(0.006168)) + 4 * (-log(0.001451)) + 5 * (-log(0.001701)) = 169.8
```

## Unigram 구현

In [23]:
corpus = [
    "This is the Hugging Face course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('xlnet-base-cased')

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

In [24]:
# 각 단어의 출현 빈도 계산
from collections import defaultdict

word_freqs = defaultdict(int)
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

word_freqs

defaultdict(int,
            {'▁This': 3,
             '▁is': 2,
             '▁the': 1,
             '▁Hugging': 1,
             '▁Face': 1,
             '▁course.': 1,
             '▁chapter': 1,
             '▁about': 1,
             '▁tokenization.': 1,
             '▁section': 1,
             '▁shows': 1,
             '▁several': 1,
             '▁tokenizer': 1,
             '▁algorithms.': 1,
             '▁Hopefully,': 1,
             '▁you': 1,
             '▁will': 1,
             '▁be': 1,
             '▁able': 1,
             '▁to': 1,
             '▁understand': 1,
             '▁how': 1,
             '▁they': 1,
             '▁are': 1,
             '▁trained': 1,
             '▁and': 1,
             '▁generate': 1,
             '▁tokens.': 1})

그리고 최종적으로 원하는 크기보다 더 크게 vocabulary를 초기화해야 합니다. 초기화 과정에서 vocabulary에 모든 기본 문자들을 포함해야 합니다. 또한 길이가 더 긴 부분 문자열에 대해서는 가장 빈번하게 출현하는 것들만 추가할 것이므로 일단 빈도순으로 정렬을 수행합니다.

In [25]:
char_freqs = defaultdict(int)
subwords_freqs = defaultdict(int)
for word, freq in word_freqs.items():
    for i in range(len(word)):
        char_freqs[word[i]] += freq
        # 길이가 적어도 2 이상인 subword들을 추가함
        for j in range(i+1, len(word)+1):
            subwords_freqs[word[i:j]] += freq

# Subword들을 빈도 역순으로 정렬
sorted_subwords = sorted(subwords_freqs.items(), key=lambda x : x[1], reverse=True)
sorted_subwords[:10]

[('▁', 31),
 ('e', 21),
 ('t', 14),
 ('i', 13),
 ('s', 13),
 ('o', 13),
 ('a', 12),
 ('n', 11),
 ('h', 9),
 ('r', 9)]

In [26]:
token_freqs = list(char_freqs.items()) + sorted_subwords[: 300-len(char_freqs)]
token_freqs = {token : freq for token, freq in token_freqs}

# SentencePiece는 ESA(Enhanced Suffix Array)라는 보다 효율적인 알고리즘을 사용하여 초기 어휘를 생성합니다.

다음으로 모든 빈도의 합을 계산하여 빈도를 확률로 변환합니다. 우리 모델의 경우 확률의 로그값을 저장할 것입니다. 작은 숫자를 곱하는 것보다 로그를 더하는 것이 수치적으로 더 안정적이기 때문입니다. 이렇게 하면 모델 손실 계산이 단순화됩니다.

In [27]:
from math import log

total_sum = sum([freq for token, freq in token_freqs.items()])
model = {token : -log(freq / total_sum) for token, freq in token_freqs.items()}

In [39]:
def encode_word(word, model):
    best_segmentations = [{"start": 0, "score": 1}] + [
        {"start": None, "score": None} for _ in range(len(word))
    ]
    for start_idx in range(len(word)):
        # This should be properly filled by the previous steps of the loop
        best_score_at_start = best_segmentations[start_idx]["score"]
        for end_idx in range(start_idx + 1, len(word) + 1):
            token = word[start_idx:end_idx]
            if token in model and best_score_at_start is not None:
                score = model[token] + best_score_at_start
                # If we have found a better segmentation ending at end_idx, we update
                if (
                    best_segmentations[end_idx]["score"] is None
                    or best_segmentations[end_idx]["score"] > score
                ):
                    best_segmentations[end_idx] = {"start": start_idx, "score": score}

    segmentation = best_segmentations[-1]
    if segmentation["score"] is None:
        # We did not find a tokenization of the word -> unknown
        return ["<unk>"], None

    score = segmentation["score"]
    start = segmentation["start"]
    end = len(word)
    tokens = []
    while start != 0:
        tokens.insert(0, word[start:end])
        next_start = best_segmentations[start]["start"]
        end = start
        start = next_start
    tokens.insert(0, word[start:end])
    return tokens, score

In [40]:
print(encode_word('Hopefully', model))
print(encode_word("This", model))

(['H', 'o', 'p', 'e', 'f', 'u', 'll', 'y'], 41.19982858248713)
(['This'], 6.2487769209879005)


In [41]:
def compute_loss(model):
    loss = 0
    for word, freq in word_freqs.items():
        _, word_loss = encode_word(word, model)
        loss += freq * word_loss
    return loss

compute_loss(model)

434.111070693413

In [43]:
import copy

def compute_scores(model):
    scores = {}
    model_loss = compute_loss(model)
    for token, score in model.items():
        # We always keep tokens of length 1
        if len(token) == 1:
            continue
        model_without_token = copy.deepcopy(model)
        _ = model_without_token.pop(token)
        scores[token] = compute_loss(model_without_token) - model_loss
    return scores

scores = compute_scores(model)
print(scores["ll"])
print(scores["his"])

6.297432184210663
0.0


In [44]:
percent_to_remove = 0.1
while len(model) > 100:
    scores = compute_scores(model)
    sorted_scores = sorted(scores.items(), key=lambda x : x[1])
    # Remove percent_to_remove tokens with the lowest scores.
    for i in range(int(len(model) * percent_to_remove)):
        _ = token_freqs.pop(sorted_scores[i][0])
    
    total_sum = sum([freq for token, freq in token_freqs.items()])
    model = {token : -log(freq / total_sum) for token, freq in token_freqs.items()}


def tokenize(text, model):
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in words_with_offsets]
    encoded_words = [encode_word(word, model)[0] for word in pre_tokenized_text]
    return sum(encoded_words, [])


tokenize('This is the Hugging Face course.', model)

['▁This', '▁is', '▁the', '▁Hugging', '▁Face', '▁course.']